In [198]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [199]:
import imblearn
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE, ADASYN

In [200]:
df = pd.read_csv('nlp_hackathon_bd_2023/data/train.txt', sep=' _ _ ', header=None)
test = pd.read_csv('nlp_hackathon_bd_2023/data/dev.txt', sep=' _ _ ', header=None)
mainTest = pd.read_csv('test.txt', sep=' _ _ ', header=None)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [201]:
mainTest.tail()

,0
152437,বিজ্ঞাপনগুলি
152438,প্রচার
152439,করা
152440,শুরু
152441,করে।


In [202]:
storeSpaceIndex = []
with open('test.txt', 'r') as file:
    i = 0
    for line in file:
        if line == '\n':
            storeSpaceIndex.append(i)
        i += 1

In [203]:
dict = {}
key = 0
for i in df.index:
    if df[0][i] not in dict:
        dict[df[0][i]] = key
        key += 1

In [204]:
df[0] = df[0].map(dict)
test[0] = test[0].map(dict)
mainTest[0] = mainTest[0].map(dict)

In [205]:
labelMapping = {'B-CORP':2, 'I-PROD':1, 'B-GRP':3, 'I-GRP':3, 'B-PROD':1, 'I-CW':5, 
                 'I-PER':6, 'B-LOC':4, 'I-CORP':2, 'O':0, 'B-PER':6, 'I-LOC':4, 'B-CW':5}
df[1] = df[1].map(labelMapping)
test[1] = test[1].map(labelMapping)

In [206]:
test[0] = test[0].replace(np.nan, -1)
mainTest[0] = mainTest[0].replace(np.nan, -1)

In [207]:
X, y = pd.DataFrame(df[0]), pd.DataFrame(df[1])
testX, testy =  pd.DataFrame(test[0]),  pd.DataFrame(test[1])
mainTestX = pd.DataFrame(mainTest[0])

In [208]:
clf = RandomForestClassifier()
rdclf = clf.fit(X, y)
y_pred = rdclf.predict(testX)
y_pred_main = rdclf.predict(mainTestX)
pd.DataFrame(classification_report(testy, y_pred, output_dict=True))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


,0,1,2,3,4,5,6,accuracy,macro avg,weighted avg
precision,0.914641,0.665158,0.652695,0.666667,0.563107,0.483516,0.672566,0.885996,0.659764,0.867545
recall,0.978160,0.460815,0.437751,0.523256,0.358025,0.156584,0.469136,0.885996,0.483389,0.885996
f1-score,0.945335,0.544444,0.524038,0.586319,0.437736,0.236559,0.552727,0.885996,0.546737,0.871311
support,8654.000000,319.000000,249.000000,344.000000,162.000000,281.000000,324.000000,0.885996,10333.000000,10333.000000


In [222]:
y_pred_main = rdclf.predict(mainTestX)

In [224]:
mappingBack = {0:'O', 1:'PROD', 2:'CORP', 3:'GRP', 4:'LOC', 5:'CW', 6:'PER'}

y_pred_main = pd.DataFrame(y_pred_main)[0].map(mappingBack)

In [226]:
final_pred = []
spaceI = 0
indexError = 0
if y_pred_main[0] == 'O':
    final_pred.append(y_pred_main[0]+'\n')
else:
    final_pred.append('B-'+y_pred_main[0]+'\n')
    
for i in range(1,len(y_pred_main)):
    while spaceI<len(storeSpaceIndex) and i+indexError==storeSpaceIndex[spaceI]:
        final_pred.append('\n')
        spaceI += 1
        indexError += 1
    if y_pred_main[i] != 'O':
        if y_pred_main[i] == y_pred_main[i-1]:
            final_pred.append('I-'+ y_pred_main[i]+'\n')
        else:
            final_pred.append('B-'+ y_pred_main[i]+'\n')
    else:
        final_pred.append(y_pred_main[i]+'\n')

In [227]:
with open('ouput.txt','w') as file:
    for e in final_pred:
        file.write(e)